<a href="https://colab.research.google.com/github/vitoriabtriz/chatbot_coord_ensino_ufpe/blob/main/CRIACOMP_Assistente_Virtual_com_OpenAI_Responses_e_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q openai gradio tdqm

In [ ]:
from google.colab import userdata
from openai import OpenAI
from tqdm import tqdm
import gradio as gr
import concurrent
import os

In [ ]:
#cria a pasta onde o pdf vai ser armazenado
!mkdir -p /content/input_pdfs


In [ ]:
from google.colab import files
uploaded = files.upload()

#move o pdf enviado para a pasta correta
import shutil
for filename in uploaded.keys():
    shutil.move(filename, f"/content/input_pdfs/{filename}")


In [ ]:
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
dir_pdfs = '/content/input_pdfs'
pdf_files = [os.path.join(dir_pdfs, f) for f in os.listdir(dir_pdfs)]

In [ ]:
def upload_single_pdf(file_path: str, vector_store_id: str):
    file_name = os.path.basename(file_path)
    try:
        file_response = client.files.create(file=open(file_path, 'rb'), purpose="assistants")
        attach_response = client.vector_stores.files.create(
            vector_store_id=vector_store_id,
            file_id=file_response.id
        )
        return {"file": file_name, "status": "success"}
    except Exception as e:
        print(f"Error with {file_name}: {str(e)}")
        return {"file": file_name, "status": "failed", "error": str(e)}

def upload_pdf_files_to_vector_store(vector_store_id: str, pdf_files: list):
    stats = {"total_files": len(pdf_files), "successful_uploads": 0, "failed_uploads": 0, "errors": []}

    print(f"{len(pdf_files)} PDF files to process. Uploading in parallel...")

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(upload_single_pdf, file_path, vector_store_id): file_path for file_path in pdf_files}
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(pdf_files)):
            result = future.result()
            if result["status"] == "success":
                stats["successful_uploads"] += 1
            else:
                stats["failed_uploads"] += 1
                stats["errors"].append(result)

    return stats

def create_vector_store(store_name: str) -> dict:
    try:
        vector_store = client.vector_stores.create(name=store_name)
        details = {
            "id": vector_store.id,
            "name": vector_store.name,
            "created_at": vector_store.created_at,
            "file_count": vector_store.file_counts.completed
        }
        print("Vector store created:", details)
        return details
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return {}

In [ ]:
store_name = "coordenacao_ensino"
vector_store_details = create_vector_store(store_name)
upload_pdf_files_to_vector_store(vector_store_details["id"], pdf_files)

In [ ]:
import concurrent.futures

def response_output(query, history):
    def make_request():
        return client.responses.create(
            model="gpt-4o-mini",
            input=[
                {
                    "role": "system",
                    "content": "Você é um assistente virtual que responde apenas com base nos documentos fornecidos sobre a coordenação do CIn/UFPE. Caso a pergunta não tenha relação com os dados, responda: 'Desculpe, não sei informar sobre isso.'"
                },
                {"role": "user", "content": query}
            ],
            tools=[
                {
                    "type": "file_search",
                    "vector_store_ids": [vector_store_details['id']],
                }
            ]
        )

    try:
        #executa com timeout de 30 segundos
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(make_request)
            response = future.result(timeout=30)

        #verifica se veio resposta relevante de um arquivo
        if (
            hasattr(response, "output")
            and len(response.output) > 1
            and hasattr(response.output[1], "content")
            and response.output[1].content
            and response.output[1].content[0].text.strip() != ""
        ):
            return response.output[1].content[0].text.strip()
        else:
            return "Desculpe, não sei informar sobre isso."

    except concurrent.futures.TimeoutError:
        return "Desculpe, não sei informar sobre isso."
    except Exception as e:
        #você pode logar o erro se quiser: print(str(e))
        return "Desculpe, não sei informar sobre isso."


In [ ]:
demo = gr.ChatInterface(
    response_output,
    type="messages"
)

demo.launch(debug=True)